# Værdata fra Meterologisk institutt

## Elementer som kan hentes i fra "observations"
### **1. Nedbør (Precipitation)**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `precipitation_amount` | Nedbørsmengde per tidsperiode | mm |
| `precipitation_amount_hourly` | Nedbør per time | mm |
| `sum(precipitation_amount PT1H)` | Summering av nedbør per time | mm |

### **2. Temperatur**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `air_temperature` | Lufttemperatur | °C |
| `air_temperature_max` | Maksimum temperatur | °C |
| `air_temperature_min` | Minimum temperatur | °C |
| `dew_point_temperature` | Duggpunkt | °C |

### **3. Vind**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `wind_speed` | Vindhastighet | m/s |
| `wind_speed_max` | Maksimal vindhastighet | m/s |
| `wind_from_direction` | Vindretning | grader (°) |
| `wind_speed_mean` | Gjennomsnittlig vindhastighet | m/s |

### **4. Lufttrykk**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `air_pressure_at_sea_level` | Lufttrykk ved havnivå | hPa |

### **5. Skyer og sikt**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `cloud_area_fraction` | Skydekkeandel | % |
| `fog_area_fraction` | Tåkeandel | % |
| `visibility` | Sikt | m |

### **6. Snø**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `snow_depth` | Snødybde | cm |

### **7. Stråling**
| Element | Beskrivelse | Enhet |
|---------|------------|-------|
| `shortwave_radiation` | Kortbølget stråling | W/m² |

## OPPGAVE
Ut i fra tabellen ovenfor, så utvider du koden til å hente flere data fra Frost API'et.
Det dere skal få til er skydekke og regn. Det er flere variabler for regn, prøv de forskjellige.
Lag grafer for hver av datatypene.

Det er mange verstasjoner å velge i Frost. Ikke alle har faktiske data. Kan du finne en lokal stasjon der du bor? 
(Bruk ChatGPT eller gå til YR og se på siden "Andre varsler", der vil du finne stasjoner i nærheten. Klikk på din målestasjon og se på URL'en. Eks.: https://www.yr.no/nb/historikk/graf/5-30330/Norge/Telemark/Skien/Gjerpen%20-%20%C3%85rhus
Her ser dere 30330 og i koden så står det SN30330. Det nummert vil da være unikt for din stasjon.)



In [8]:
import pandas as pd

df = pd.read_csv("Renset_CSV-data.csv")
df.head()

,Tidspunkt,Luftfuktighet (%),pressure,temperature_aht20,temperature_bmp280,tvoc
0,2025-02-11 09:00:00+00:00,23.058358,103.81045,19.384174,19.064306,1.1
1,2025-02-24 08:00:00+00:00,38.309145,100.56495,22.579100,22.313900,12.0
2,2025-02-24 09:00:00+00:00,46.899600,100.53990,16.988750,16.633570,0.0
3,2025-02-24 10:00:00+00:00,48.069380,100.51740,16.487120,16.145930,0.0
4,2025-02-24 11:00:00+00:00,48.081020,100.50180,16.729160,16.391020,0.0


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# === Frost API Konfigurasjon ===
CLIENT_ID = "946fbb54-4c6d-4e38-9e4b-0dff67d726ad"
station_id = 'SN30330'  # GJERPEN - ÅRHUS
station_name = 'GJERPEN - ÅRHUS (ID: SN30330)'
start_date = "2025-02-01"
end_date = "2025-02-28"

# === CSV-fil Konfigurasjon ===
csv_file_path = "Renset_CSV-data.csv"

def get_frost_data(station_id, start_date, end_date):
    elements = "air_temperature,wind_speed,relative_humidity,precipitation_amount"
    url = f"https://frost.met.no/observations/v0.jsonld?sources={station_id}&referencetime={start_date}/{end_date}&elements={elements}"
    response = requests.get(url, auth=(CLIENT_ID, ""))
    data = response.json()

    records = []
    for item in data.get("data", []):
        timestamp = item["referenceTime"]
        values = {"Tidspunkt": timestamp}
        
        for o in item["observations"]:
            values[o["elementId"]] = o["value"]
        
        records.append(values)

    df = pd.DataFrame(records)
    df["Tidspunkt"] = pd.to_datetime(df["Tidspunkt"]).dt.tz_localize(None)
    return df

def get_csv_data(file_path):
    df_csv = pd.read_csv(file_path)
    df_csv["Tidspunkt"] = pd.to_datetime(df_csv["Tidspunkt"], errors="coerce")
    df_csv = df_csv.dropna(subset=["Tidspunkt"])
    df_csv["Tidspunkt"] = df_csv["Tidspunkt"].dt.tz_localize(None)  # Fjern tidssone for samsvar med Frost API
    return df_csv

# === Hent data ===
df_frost = get_frost_data(station_id, start_date, end_date)
df_csv = get_csv_data(csv_file_path)

# === Slå sammen datasettene basert på nærmeste tid ===
df_combined = pd.merge_asof(df_csv.sort_values("Tidspunkt"), 
                            df_frost.sort_values("Tidspunkt"), 
                            on="Tidspunkt", 
                            direction="nearest")

# === Plot temperatur ===
plt.figure(figsize=(12, 5))
plt.plot(df_combined["Tidspunkt"], df_combined["temperature_aht20"], marker="o", linestyle="-", label="Sensor AHT20 (CSV)", color="b")
plt.plot(df_combined["Tidspunkt"], df_combined["temperature_bmp280"], marker="s", linestyle="-", label="Sensor BMP280 (CSV)", color="g")
plt.plot(df_combined["Tidspunkt"], df_combined["air_temperature"], marker="x", linestyle="--", label="Frost API", color="r")
plt.xlabel("Dato")
plt.ylabel("Temperatur (°C)")
plt.title(f"Temperatur ved {station_name}")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# === Plot luftfuktighet ===
plt.figure(figsize=(12, 5))
plt.plot(df_combined["Tidspunkt"], df_combined["Luftfuktighet (%)"], marker="o", linestyle="-", label="Sensor (CSV)", color="y")
plt.plot(df_combined["Tidspunkt"], df_combined["relative_humidity"], marker="x", linestyle="--", label="Frost API", color="c")
plt.xlabel("Dato")
plt.ylabel("Luftfuktighet (%)")
plt.title(f"Luftfuktighet ved {station_name}")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()
